## HW4.0 Final Project description

##### Project Proposal

Intelligent Tutoring Systems are helping saving millions of hours for students and helping them learn deeper. Students imbibe concepts at varying rates of learning. By predicting student performance on problems, content can be tailored to their needs. Target has trainings around the year, which are often limited by its class size. Instead, if these trainings are delivered online adaptive to the learning ability of each employee, we can create a more skilled work force and also build a knowledge repository. The aim of the project is to predict student performance from the log of online trainings for algebra (KDD Cup 2010, Educational Data Mining Challenge. 

##### Data Source and Description 

The data is from the KDD cup. Four key terms form the building blocks of the data. These are problem (tasks for the student), step (observable part of the solution), knowledge component (concept required for a step), and opportunity (chance to demonstrate learning a knowledge component).

##### Steps

1. Data and Problem Understanding
2. Build baseline model on sample 
3. EDA
4. Build model on training set 
5. Test model on a hold out set of the data

##### Success metrics Primary- 

Prediction accuracy of student performance- (AUC for this binary classification task) Secondary- Similarity measures for knowledge components, Time saved in hours 


### HW4.1 Build a decision to predict whether you can play tennis or not

###### HW4.1.1 What is the classification accuracy of the tree on the training data?

###### Ans : 
The classification accuracy of a tree on a training dataset will be 100% because an ID3 algorithm will keep on splitting the tree till it reaches the maximum number of layers or till it runs out of examples.

##### HW4.1.2 Is it possible to produce some set of correct training examples that will get the algorihtm to include the attribute Temperature in the learned tree, even though the true target concept is independent of Temperature? if no, explain. If yes, give such a set.

##### Ans. 


###### HW4.1.3 Now, build a tree using only examples D1–D7. What is the classification accuracy for the training set? what is the accuracy for the test set (examples D8–D14)? explain why you think these are the results.

###### Ans. 
The accuracy in the training dataset (D1-D7) is 100%.

##### HW4.1.4 In this case, and others, there are only a few labelled examples available for training (that is, no additional data is available for testing or validation). Suggest a concrete pruning strategy, that can be readily embedded in the algorithm, to avoid over fitting. Explain why you think this strategy should work.

##### Ans. 
overfitting can be avoided using decision tree pruning or by limiting the depth of the tree.

### Decision Trees for Classification ID3

In [24]:
%%writefile dataset.txt
d1 sunny hot high FALSE no
d2 sunny hot high TRUE no
d3 overcast hot high FALSE yes
d4 rainy mild high FALSE yes
d5 rainy cool normal FALSE yes
d6 rainy cool normal TRUE no
d6 overcast cool normal TRUE yes
d7 sunny mild high FALSE no
d8 sunny cool normal FALSE yes
d9 rainy mild normal FALSE yes
d10 sunny mild normal TRUE yes
d11 overcast mild high TRUE yes
d12 overcast hot normal FALSE yes
d12 rainy mild high TRUE no

Overwriting dataset.txt


In [25]:
f=open("/Users/z00193k/Desktop/DataScienceTrainingDocs/Assignments/DecisionTree/dataset.txt", "r")
inputs=[]
for line in f:
    inter=[]
    current={}
    splitline=[]
    splitline = line.split()
    current["outlook"]=splitline[1]
    current["temperature"]=splitline[2]
    current["humidity"]=splitline[3]
    current["wind"]=splitline[4]
    inter.append(current)
    inter.append(splitline[5])
    inter=tuple(inter)
    inputs.append(inter)

In [26]:
#calculating the entropy of the data

from __future__ import division
from collections import Counter, defaultdict
from functools import partial
import math, random

def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2) for p in class_probabilities if p)

def class_probabilities(labels):
    total_count = len(labels)
    return [count/total_count for count in Counter(labels).values()]

#This function will return the entropy of input dataset
def data_entropy(labeled_data):        
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

In [27]:
data_entropy(inputs)

0.9402859586706309

### Entropy of a Partition

In [28]:
def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)    
    return sum(data_entropy(subset)*len(subset)/total_count for subset in subsets )

In [31]:
def group_by(items, key_fn):
    """returns a defaultdict(list), where each input item 
    is in the list whose key is key_fn(item)"""
    groups = defaultdict(list)
    for item in items:
        key = key_fn(item)
        groups[key].append(item)
    return groups
    
def partition_by(inputs, attribute):
    """returns a dict of inputs partitioned by the attribute
    each input is a pair (attribute_dict, label)"""
    return group_by(inputs, lambda x: x[0][attribute])

def partition_entropy_by(inputs, attribute):
    """computes the entropy corresponding to the given partition"""        
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())

In [32]:
for key in['outlook','temperature','humidity','wind']:
    print key, partition_entropy_by(inputs, key)

outlook 0.693536138896
temperature 0.911063393012
humidity 0.788450457308
wind 0.892158928262


### Information Gain

In [33]:
attributes = ['outlook','temperature','humidity','wind']
def inf_gain(inputs, keys):
    for key in keys:
        IG=data_entropy(inputs)-partition_entropy_by(inputs, key)
        print key, IG

inf_gain(inputs, attributes)

outlook 0.246749819774
temperature 0.029222565659
humidity 0.151835501362
wind 0.0481270304083


In [34]:
def classify(tree, input):
    """classify the input using the given decision tree"""
    
    # if this is a leaf node, return its value
    if tree in [True, False]:
        return tree
   
    # otherwise find the correct subtree
    attribute, subtree_dict = tree
    
    subtree_key = input.get(attribute)  # None if input is missing attribute

    if subtree_key not in subtree_dict: # if no subtree for key,
        subtree_key = None              # we'll use the None subtree
    
    subtree = subtree_dict[subtree_key] # choose the appropriate subtree
    return classify(subtree, input)     # and use it to classify the input

In [35]:
def build_tree_id3(inputs, split_candidates=None):

    # if this is our first pass, 
    # all keys of the first input are split candidates
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()

    # count Trues and Falses in the inputs
    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues
    
    if num_trues == 0:                  # if only Falses are left
        return False                    # return a "False" leaf
        
    if num_falses == 0:                 # if only Trues are left
        return True                     # return a "True" leaf

    if not split_candidates:            # if no split candidates left
        return num_trues >= num_falses  # return the majority leaf
                            
    # otherwise, split on the best attribute
    best_attribute = min(split_candidates,
        key=partial(partition_entropy_by, inputs))

    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates 
                      if a != best_attribute]
    
    # recursively build the subtrees
    subtrees = { attribute : build_tree_id3(subset, new_candidates)
                 for attribute, subset in partitions.iteritems() }

    subtrees[None] = num_trues > num_falses # default case

    return (best_attribute, subtrees)

print "building the tree"
tree = build_tree_id3(inputs)
print tree

building the tree
True
